In [ ]:
# default_exp tst_net

In [ ]:
# #hide
# # from nbdev.showdoc import *
# from fastcore.test import *

In [ ]:
# # export
# import torch.nn as nn
# from collections import OrderedDict
# from model_constructor.layers import *

In [ ]:
# # export
# import torch,math,sys
# import torch.utils.model_zoo as model_zoo
# from functools import partial
# # from fastai.torch_core import Module

# Xresnet from fastai v1

## helpers

In [ ]:
# # export
# act_fn = nn.ReLU(inplace=True)

# # in layers
# class Flatten(Module):
#     def forward(self, x): return x.view(x.size(0), -1)

# def init_cnn(m):
#     if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
#     if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
#     for l in m.children(): init_cnn(l)

# def conv(ni, nf, ks=3, stride=1, bias=False):
#     return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)
# # in layers
# def noop(x): return x

## conv layer

In [ ]:
# # export

# def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, act=True):
#     bn = nn.BatchNorm2d(nf)
#     nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
#     layers = [conv(ni, nf, ks, stride=stride), bn]
#     if act: layers.append(act_fn)
#     return nn.Sequential(*layers)

## ResBlock

In [ ]:
# # v1
# class ResBlock(Module):
#     def __init__(self, expansion, ni, nh, stride=1):
#         nf,ni = nh*expansion,ni*expansion
#         layers  = [conv_layer(ni, nh, 3, stride=stride),
#                    conv_layer(nh, nf, 3, zero_bn=True, act=False)
#         ] if expansion == 1 else [
#                    conv_layer(ni, nh, 1),
#                    conv_layer(nh, nh, 3, stride=stride),
#                    conv_layer(nh, nf, 1, zero_bn=True, act=False)
#         ]
#         self.convs = nn.Sequential(*layers)
#         # TODO: check whether act=True works better
#         self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
#         self.pool = noop if stride==1 else nn.AvgPool2d(2, ceil_mode=True)

#     def forward(self, x): return act_fn(self.convs(x) + self.idconv(self.pool(x)))


In [ ]:
# # v2
# class ResBlock(Module):
#     def __init__(self, expansion, ni, nh, stride=1, 
#                  conv_layer=conv_layer, act_fn=act_fn,
#                  pool=nn.AvgPool2d(2, ceil_mode=True)):
#         nf,ni = nh*expansion,ni*expansion
#         layers  = [(f"conv_0", conv_layer(ni, nh, 3, stride=stride)),
#                    (f"conv_1", conv_layer(nh, nf, 3, zero_bn=True, act=False))
#         ] if expansion == 1 else [
#                    (f"conv_0",conv_layer(ni, nh, 1)),
#                    (f"conv_1",conv_layer(nh, nh, 3, stride=stride)),
#                    (f"conv_2",conv_layer(nh, nf, 1, zero_bn=True, act=False))
#         ]
#         self.convs = nn.Sequential(OrderedDict(layers))
#         self.pool = noop if stride==1 else pool
#         self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
#         self.act_fn =act_fn

#     def forward(self, x): return self.act_fn(self.convs(x) + self.idconv(self.pool(x)))

In [ ]:
# # export
# class ResBlock(Module):
#     def __init__(self, expansion, ni, nh, stride=1, 
#                  conv_layer=conv_layer, act_fn=act_fn,
#                  pool=nn.AvgPool2d(2, ceil_mode=True), sa=False,sym=False):
#         nf,ni = nh*expansion,ni*expansion
#         layers  = [(f"conv_0", conv_layer(ni, nh, 3, stride=stride)),
#                    (f"conv_1", conv_layer(nh, nf, 3, zero_bn=True, act=False))
#         ] if expansion == 1 else [
#                    (f"conv_0",conv_layer(ni, nh, 1)),
#                    (f"conv_1",conv_layer(nh, nh, 3, stride=stride)),
#                    (f"conv_2",conv_layer(nh, nf, 1, zero_bn=True, act=False))
#         ]
#         if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
#         self.convs = nn.Sequential(OrderedDict(layers))
#         self.pool = noop if stride==1 else pool
#         self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
#         self.act_fn =act_fn

#     def forward(self, x): return self.act_fn(self.convs(x) + self.idconv(self.pool(x)))

In [ ]:
# ResBlock(1,64,64,sa=True)

In [ ]:
# ResBlock(1,64,64,)

## first vers XResNet

In [ ]:
# # initial version
# class XResNet(nn.Sequential):
#     def __init__(self, expansion, layers, c_in=3, c_out=1000):
#         stem = []
#         sizes = [c_in,32,32,64]
#         for i in range(3):
#             stem.append(conv_layer(sizes[i], sizes[i+1], stride=2 if i==0 else 1))
          
#         block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
#         blocks = [self._make_layer(expansion, block_szs[i], block_szs[i+1], l, 1 if i==0 else 2)
#                   for i,l in enumerate(layers)]
#         super().__init__(
#             *stem,
#             nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
#             *blocks,
#             nn.AdaptiveAvgPool2d(1), Flatten(),
#             nn.Linear(block_szs[-1]*expansion, c_out),
#         )
#         init_cnn(self)

#     def _make_layer(self, expansion, ni, nf, blocks, stride):
#         return nn.Sequential(
#             *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1)
#               for i in range(blocks)])

In [ ]:
# # v2
# class XResNet(nn.Sequential):
#     def __init__(self, expansion, layers, c_in=3, c_out=1000):
#         stem = []
#         sizes = [c_in,32,32,64]
#         for i in range(3):
#             stem.append(conv_layer(sizes[i], sizes[i+1], stride=2 if i==0 else 1))
#              block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
#         blocks = [self._make_layer(expansion, block_szs[i], block_szs[i+1], l, 1 if i==0 else 2)
#                   for i,l in enumerate(layers)]
#         super().__init__(OrderedDict([
#             ('stem', nn.Sequential(*stem, nn.MaxPool2d(kernel_size=3, stride=2, padding=1))),
#             ('body', nn.Sequential(*blocks)),
#             ('head', nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten(), 
#              nn.Linear(block_szs[-1]*expansion, c_out)))
#         ]))
#         init_cnn(self)

#     def _make_layer(self, expansion, ni, nf, blocks, stride):
#         return nn.Sequential(
#             *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1)
#               for i in range(blocks)])

In [ ]:
# # v3
# class XResNet():
#     def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000):
#         super().__init__()
#         self.c_in, self.c_out,self.expansion = c_in,c_out,expansion
#         self.stem = []
#         sizes = [c_in,32,32,64]
#         for i in range(3):
#             self.stem.append(conv_layer(sizes[i], sizes[i+1], stride=2 if i==0 else 1))
#         self.block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
#         self.blocks = [self._make_layer(expansion, self.block_szs[i], self.block_szs[i+1], l, 1 if i==0 else 2)
#                   for i,l in enumerate(layers)]
        

#     def _make_layer(self, expansion, ni, nf, blocks, stride):
#         return nn.Sequential(
#             *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1)
#               for i in range(blocks)])
#     def __call__(self):
#         model = nn.Sequential(OrderedDict([
#             ('stem', nn.Sequential(*self.stem, nn.MaxPool2d(kernel_size=3, stride=2, padding=1))),
#             ('body', nn.Sequential(*self.blocks)),
#             ('head', nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten(), 
#              nn.Linear(self.block_szs[-1]*self.expansion, self.c_out)))
#         ]))
#         init_cnn(model)
#         return model

In [ ]:
# # v4
# class XResNet():
#     def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000):
#         super().__init__()
#         self.c_in, self.c_out,self.expansion = c_in,c_out,expansion
#         self.stem = []
#         sizes = [c_in,32,32,64]
#         for i in range(3):
#             self.stem.append(conv_layer(sizes[i], sizes[i+1], stride=2 if i==0 else 1))
#         self.stem.append(nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
#         block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
#         self.blocks = [self._make_layer(expansion, block_szs[i], block_szs[i+1], l, 1 if i==0 else 2)
#                   for i,l in enumerate(layers)]
#         self.head = [nn.AdaptiveAvgPool2d(1), Flatten(), 
#              nn.Linear(block_szs[-1]*self.expansion, self.c_out)]

#     def _make_layer(self, expansion, ni, nf, blocks, stride):
#         return nn.Sequential(
#             *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1)
#               for i in range(blocks)])
#     def __call__(self):
#         model = nn.Sequential(OrderedDict([
#             ('stem', nn.Sequential(*self.stem)),
#             ('body', nn.Sequential(*self.blocks)),
#             ('head', nn.Sequential(*self.head))
#         ]))
#         init_cnn(model)
#         return model

In [ ]:
# # v5
# class XResNet():
#     def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000):
#         super().__init__()
#         self.c_in, self.c_out,self.expansion,self.layers = c_in,c_out,expansion,layers
#         self.stem_sizes = [c_in,32,32,64]
#         self.block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
#         self.block = ResBlock
#         self.norm = nn.BatchNorm2d
#         self.act_fn=nn.ReLU(inplace=True)
# #         self._conv_layer = conv_layer
    
# #     @property
# #     def conv_layer(self): return self._conv_layer

#     def conv_layer(self, ni, nf, ks=3, stride=1, zero_bn=False, act=True):
#         bn = self.norm(nf)
#         nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
#         layers = [('conv', conv(ni, nf, ks, stride=stride)), ('norm', bn)]
#         if act: layers.append(('act_fn', self.act_fn))
#         return nn.Sequential(OrderedDict(layers))
    
#     @property
#     def stem(self):
#         return self._make_stem()
#     @property
#     def head(self):
#         return self._make_head()
#     @property
#     def body(self):
#         return self._make_body()
    
#     def _make_stem(self):
#         stem = []
#         for i in range(len(self.stem_sizes)-1):
#             stem.append((f"conv_{i}", self.conv_layer(self.stem_sizes[i], self.stem_sizes[i+1], stride=2 if i==0 else 1)))
#         stem.append(('stem_pool', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)))
#         return nn.Sequential(OrderedDict(stem))
    
#     def _make_head(self):
#         head = [('pool', nn.AdaptiveAvgPool2d(1)),
#                 ('flat', Flatten()),
#                 ('fc',   nn.Linear(self.block_szs[-1]*self.expansion, self.c_out))]
#         return nn.Sequential(OrderedDict(head))
    
#     def _make_body(self):
#         blocks = [(f"l_{i}", self._make_layer(self.expansion, 
#                         self.block_szs[i], self.block_szs[i+1], l, 1 if i==0 else 2))
#                   for i,l in enumerate(self.layers)]
#         return nn.Sequential(OrderedDict(blocks))
    
#     def _make_layer(self, expansion, ni, nf, blocks, stride):
#         return nn.Sequential(
#             *[self.block(expansion, ni if i==0 else nf, nf, 
#                          stride if i==0 else 1,conv_layer=self.conv_layer)
#               for i in range(blocks)])
#     def __call__(self):
#         model = nn.Sequential(OrderedDict([
# #             ('stem', nn.Sequential(*self.stem)),
#             ('stem', self.stem),
#             ('body', self.body),
#             ('head', self.head)
#         ]))
#         init_cnn(model)
#         return model

In [ ]:
# # v6
# class XResNet():
#     def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000):
#         super().__init__()
#         self.c_in, self.c_out,self.expansion,self.layers = c_in,c_out,expansion,layers
#         self.stem_sizes = [c_in,32,32,64]
#         self.block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
#         self.block = ResBlock
#         self.norm = nn.BatchNorm2d
#         self.act_fn=nn.ReLU(inplace=True)
#         self.stem_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.pool = nn.AvgPool2d(2, ceil_mode=True)
#         self.sa=False
# #         self._conv_layer = conv_layer
    
# #     @property
# #     def conv_layer(self): return self._conv_layer

#     def conv_layer(self, ni, nf, ks=3, stride=1, zero_bn=False, act=True):
#         bn = self.norm(nf)
#         nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
#         layers = [('conv', conv(ni, nf, ks, stride=stride)), ('norm', bn)]
#         if act: layers.append(('act_fn', self.act_fn))
#         return nn.Sequential(OrderedDict(layers))
    
#     @property
#     def stem(self):
#         return self._make_stem()
#     @property
#     def head(self):
#         return self._make_head()
#     @property
#     def body(self):
#         return self._make_body()
    
#     def _make_stem(self):
#         stem = []
#         for i in range(len(self.stem_sizes)-1):
#             stem.append((f"conv_{i}", self.conv_layer(self.stem_sizes[i], self.stem_sizes[i+1], stride=2 if i==0 else 1)))
#         stem.append(('stem_pool', self.stem_pool))
#         return nn.Sequential(OrderedDict(stem))
    
#     def _make_head(self):
#         head = [('pool', nn.AdaptiveAvgPool2d(1)),
#                 ('flat', Flatten()),
#                 ('fc',   nn.Linear(self.block_szs[-1]*self.expansion, self.c_out))]
#         return nn.Sequential(OrderedDict(head))
    
#     def _make_body(self):
#         blocks = [(f"l_{i}", self._make_layer(self.expansion, 
#                         self.block_szs[i], self.block_szs[i+1], l, 
#                         1 if i==0 else 2, self.sa if i==0 else False))
#                   for i,l in enumerate(self.layers)]
#         return nn.Sequential(OrderedDict(blocks))
    
#     def _make_layer(self, expansion, ni, nf, blocks, stride,sa):
#         return nn.Sequential(OrderedDict(
#             [(f"bl_{i}", self.block(expansion, ni if i==0 else nf, nf, 
#                     stride if i==0 else 1, sa=sa if i==blocks-1 else False,
#                     conv_layer=self.conv_layer, act_fn=self.act_fn, pool=self.pool))
#               for i in range(blocks)]))
#     def __call__(self):
#         model = nn.Sequential(OrderedDict([
# #             ('stem', nn.Sequential(*self.stem)),
#             ('stem', self.stem),
#             ('body', self.body),
#             ('head', self.head)
#         ]))
#         init_cnn(model)
#         return model

# XResNet

In [ ]:
# # export
# # v6
# class XResNet():
#     def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000):
#         super().__init__()
#         self.c_in, self.c_out,self.expansion,self.layers = c_in,c_out,expansion,layers
#         self.stem_sizes = [c_in,32,32,64]
#         self.stem_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.stem_bn_end = False
#         self.block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
#         self.block = ResBlock
#         self.norm = nn.BatchNorm2d
#         self.act_fn=nn.ReLU(inplace=True)
#         self.pool = nn.AvgPool2d(2, ceil_mode=True)
#         self.sa=False
#         self.bn_1st = True
#         #         self._conv_layer = conv_layer
#         #     @property
#         #     def conv_layer(self): return self._conv_layer

#     def conv_layer(self, ni, nf, ks=3, stride=1, zero_bn=False, act=True, norm=True):
#         bn = self.norm(nf)
#         nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
#         layers =  [('norm', bn)] if norm else []
#         if act: layers.append(('act_fn', self.act_fn))
#         if not self.bn_1st: layers.reverse()
#         layers = [('conv', conv(ni, nf, ks, stride=stride))] + layers
#         return nn.Sequential(OrderedDict(layers))
    
#     @property
#     def stem(self):
#         return self._make_stem()
#     @property
#     def head(self):
#         return self._make_head()
#     @property
#     def body(self):
#         return self._make_body()
    
#     def _make_stem(self):
#         stem = [(f"conv_{i}", self.conv_layer(self.stem_sizes[i], self.stem_sizes[i+1], 
#                         stride=2 if i==0 else 1, 
#                         norm=(not self.stem_bn_end) if i==(len(self.stem_sizes)-2) else True))
#                 for i in range(len(self.stem_sizes)-1)]
#         stem.append(('stem_pool', self.stem_pool))
#         if self.stem_bn_end: stem.append(('norm', self.norm(self.stem_sizes[-1])))
#         return nn.Sequential(OrderedDict(stem))
    
#     def _make_head(self):
#         head = [('pool', nn.AdaptiveAvgPool2d(1)),
#                 ('flat', Flatten()),
#                 ('fc',   nn.Linear(self.block_szs[-1]*self.expansion, self.c_out))]
#         return nn.Sequential(OrderedDict(head))
    
#     def _make_body(self):
#         blocks = [(f"l_{i}", self._make_layer(self.expansion, 
#                         self.block_szs[i], self.block_szs[i+1], l, 
#                         1 if i==0 else 2, self.sa if i==0 else False))
#                   for i,l in enumerate(self.layers)]
#         return nn.Sequential(OrderedDict(blocks))
    
#     def _make_layer(self, expansion, ni, nf, blocks, stride,sa):
#         return nn.Sequential(OrderedDict(
#             [(f"bl_{i}", self.block(expansion, ni if i==0 else nf, nf, 
#                     stride if i==0 else 1, sa=sa if i==blocks-1 else False,
#                     conv_layer=self.conv_layer, act_fn=self.act_fn, pool=self.pool))
#               for i in range(blocks)]))
    
#     def __call__(self):
#         model = nn.Sequential(OrderedDict([
#             ('stem', self.stem),
#             ('body', self.body),
#             ('head', self.head)
#         ]))
#         init_cnn(model)
#         return model

In [ ]:
# # v7
# class Net():
#     def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000, name='Net'):
#         super().__init__()
#         self.name = name
#         self.c_in, self.c_out,self.expansion,self.layers = c_in,c_out,expansion,layers
#         self.stem_sizes = [c_in,32,32,64]
#         self.stem_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.stem_bn_end = False
#         self.block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
#         self.block = ResBlock
#         self.norm = nn.BatchNorm2d
#         self.act_fn=nn.ReLU(inplace=True)
#         self.pool = nn.AvgPool2d(2, ceil_mode=True)
#         self.sa=False
#         self.bn_1st = True
#         self.zero_bn=True

#     def conv_layer(self, ni, nf, ks=3, stride=1, zero_bn=False, act=True, norm=True):
#         bn = self.norm(nf)
#         nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
#         layers =  [('norm', bn)] if norm else []
#         if act: layers.append(('act_fn', self.act_fn))
#         if not self.bn_1st: layers.reverse()
#         layers = [('conv', conv(ni, nf, ks, stride=stride))] + layers
#         return nn.Sequential(OrderedDict(layers))
    
#     @property
#     def stem(self):
#         return self._make_stem()
#     @property
#     def head(self):
#         return self._make_head()
#     @property
#     def body(self):
#         return self._make_body()
    
#     def _make_stem(self):
#         stem = [(f"conv_{i}", self.conv_layer(self.stem_sizes[i], self.stem_sizes[i+1], 
#                         stride=2 if i==0 else 1, 
#                         norm=(not self.stem_bn_end) if i==(len(self.stem_sizes)-2) else True))
#                 for i in range(len(self.stem_sizes)-1)]
#         stem.append(('stem_pool', self.stem_pool))
#         if self.stem_bn_end: stem.append(('norm', self.norm(self.stem_sizes[-1])))
#         return nn.Sequential(OrderedDict(stem))
    
#     def _make_head(self):
#         head = [('pool', nn.AdaptiveAvgPool2d(1)),
#                 ('flat', Flatten()),
#                 ('fc',   nn.Linear(self.block_szs[-1]*self.expansion, self.c_out))]
#         return nn.Sequential(OrderedDict(head))
    
#     def _make_body(self):
#         blocks = [(f"l_{i}", self._make_layer(self.expansion, 
#                         self.block_szs[i], self.block_szs[i+1], l, 
#                         1 if i==0 else 2, self.sa if i==0 else False))
#                   for i,l in enumerate(self.layers)]
#         return nn.Sequential(OrderedDict(blocks))
    
#     def _make_layer(self,expansion,ni,nf,blocks,stride,sa):
#         return nn.Sequential(OrderedDict(
#             [(f"bl_{i}", self.block(expansion, ni if i==0 else nf, nf, 
#                     stride if i==0 else 1, sa=sa if i==blocks-1 else False,
#                     conv_layer=self.conv_layer, act_fn=self.act_fn, pool=self.pool,zero_bn=self.zero_bn))
#               for i in range(blocks)]))
    
#     def __call__(self):
#         model = nn.Sequential(OrderedDict([
#             ('stem', self.stem),
#             ('body', self.body),
#             ('head', self.head)
#         ]))
#         init_cnn(model)
#         model.extra_repr = lambda : f"model {self.name}"
#         return model
#     def __repr__(self):
#         return f" constr {self.name}"

In [ ]:
# model  = XResNet()

In [ ]:
# model.stem

In [ ]:
# model.bn_1st = False

In [ ]:
# model.act_fn =nn.LeakyReLU(inplace=True)

In [ ]:
# model.sa = True

In [ ]:
# model.stem_bn_end = True

In [ ]:
# m = model()
# m

In [ ]:
# bs_test = 16
# xb = torch.randn(bs_test, 3, 128, 128)
# y = m(xb)
# # y.shape
# print(y.shape)
# assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

In [ ]:
# m.stem

In [ ]:
# m.body.l_0

In [ ]:
# model.conv_layer(64,64)

In [ ]:
# model.stem_sizes

## xresnet constructor

In [ ]:
# # 1 ver 
# def xresnet(expansion, n_layers, name, c_out=1000, pretrained=False, **kwargs):
#     model = XResNet(expansion, n_layers, c_out=c_out, **kwargs)
# #     return model
#     return model()

In [ ]:
# # v1
# me = sys.modules[__name__]
# for n,e,l in [
#     [ 18 , 1, [2,2,2 ,2] ],
#     [ 34 , 1, [3,4,6 ,3] ],
#     [ 50 , 4, [3,4,6 ,3] ],
#     [ 101, 4, [3,4,23,3] ],
#     [ 152, 4, [3,8,36,3] ],
# ]:
#     name = f'xresnet{n}'
#     setattr(me, name, partial(xresnet, expansion=e, n_layers=l, name=name))

# xresnet18_deep = partial(xresnet, expansion=1, n_layers=[2, 2,  2, 2,1,1], name='xresnet18_deep')
# xresnet34_deep = partial(xresnet, expansion=1, n_layers=[3, 4,  6, 3,1,1], name='xresnet34_deep')
# xresnet50_deep = partial(xresnet, expansion=4, n_layers=[3, 4,  6, 3,1,1], name='xresnet50_deep')


In [ ]:
# def xresnet(expansion, n_layers, name, c_out=1000, pretrained=False, **kwargs):
#     model = XResNet(expansion, n_layers, c_out=c_out, **kwargs)
#     return model

In [ ]:
# # export
# def xresnet(expansion, n_layers, name, c_out=1000, pretrained=False, **kwargs):
#     return partial(XResNet, expansion, n_layers, c_out=c_out, **kwargs)


In [ ]:
# xresnet50_d = xresnet(expansion=4, n_layers=[3, 4,  6, 3,1,1], name='xresnet50_deep')

In [ ]:
# m = xresnet50_d(c_out=10)

In [ ]:
# m.c_out

In [ ]:
# # export
# me = sys.modules[__name__]
# for n,e,l in [
#     [ 18 , 1, [2,2,2 ,2] ],
#     [ 34 , 1, [3,4,6 ,3] ],
#     [ 50 , 4, [3,4,6 ,3] ],
#     [ 101, 4, [3,4,23,3] ],
#     [ 152, 4, [3,8,36,3] ],
# ]:
#     name = f'xresnet{n}'
#     setattr(me, name, xresnet(expansion=e, n_layers=l, name=name))

# xresnet18_deep = xresnet(expansion=1, n_layers=[2, 2,  2, 2,1,1], name='xresnet18_deep')
# xresnet34_deep = xresnet(expansion=1, n_layers=[3, 4,  6, 3,1,1], name='xresnet34_deep')
# xresnet50_deep = xresnet(expansion=4, n_layers=[3, 4,  6, 3,1,1], name='xresnet50_deep')


In [ ]:
# me = sys.modules[__name__]
# for n,e,l in [[ 18 , 1, [2,2,2 ,2] ],
#     [ 34 , 1, [3,4,6 ,3] ],
#     [ 50 , 4, [3,4,6 ,3] ],
#     [ 101, 4, [3,4,23,3] ],
#     [ 152, 4, [3,8,36,3] ],]:
#     name = f'net{n}'
#     setattr(me, name, partial(XResNet, expansion=e, layers=l,))# name=name))

# end
model_constructor
by ayasyrev

In [ ]:
# # hide
# from nbdev.export import *
# notebook2script()